# Telegram

本笔记本展示了如何使用 Telegram 聊天加载器。此类有助于将导出的 Telegram 对话映射到 LangChain 聊天消息。

该过程包含三个步骤：
1. 通过从 Telegram 应用复制聊天记录，然后将它们粘贴到本地计算机上的文件中来导出聊天 .txt 文件
2. 创建 `TelegramChatLoader`，并将文件路径指向 JSON 文件或 JSON 文件目录
3. 调用 `loader.load()`（或 `loader.lazy_load()`）执行转换。可以选择使用 `merge_chat_runs` 来合并来自同一发送者的连续消息，和/或使用 `map_ai_messages` 将来自指定发送者的消息转换为 "AIMessage" 类。

## 1. 创建消息转储

目前（2023/08/23），此加载器最支持从 [Telegram Desktop App](https://desktop.telegram.org/) 导出聊天记录所生成的格式的 JSON 文件。

**重要提示：** 存在一些 Telegram 的“精简”版本，例如“Telegram for MacOS”，它们缺少导出功能。请确保您使用正确的应用程序来导出文件。

进行导出：
1. 下载并打开 Telegram Desktop
2. 选择一个对话
3. 导航到对话设置（目前是右上角的三个点）
4. 点击“导出聊天记录”
5. 取消选择照片和其他媒体。选择“机器可读的 JSON”格式进行导出。

示例如下：

In [1]:
%%writefile telegram_conversation.json
{
 "name": "Jiminy",
 "type": "personal_chat",
 "id": 5965280513,
 "messages": [
  {
   "id": 1,
   "type": "message",
   "date": "2023-08-23T13:11:23",
   "date_unixtime": "1692821483",
   "from": "Jiminy Cricket",
   "from_id": "user123450513",
   "text": "You better trust your conscience",
   "text_entities": [
    {
     "type": "plain",
     "text": "You better trust your conscience"
    }
   ]
  },
  {
   "id": 2,
   "type": "message",
   "date": "2023-08-23T13:13:20",
   "date_unixtime": "1692821600",
   "from": "Batman & Robin",
   "from_id": "user6565661032",
   "text": "What did you just say?",
   "text_entities": [
    {
     "type": "plain",
     "text": "What did you just say?"
    }
   ]
  }
 ]
}

Overwriting telegram_conversation.json


## 2. 创建聊天加载器

所需文件路径即可。您可以选择性地指定映射到 AI 消息的用户名，以及配置是否合并消息运行。

In [2]:
from langchain_community.chat_loaders.telegram import TelegramChatLoader

In [3]:
loader = TelegramChatLoader(
    path="./telegram_conversation.json",
)

## 3. 加载消息

`load()` (或 `lazy_load`) 方法会返回一个 "ChatSessions" 列表，目前该列表只包含每个已加载对话的消息列表。

In [4]:
from typing import List

from langchain_community.chat_loaders.utils import (
    map_ai_messages,
    merge_chat_runs,
)
from langchain_core.chat_sessions import ChatSession

raw_messages = loader.lazy_load()
# Merge consecutive messages from the same sender into a single message
merged_messages = merge_chat_runs(raw_messages)
# Convert messages from "Jiminy Cricket" to AI messages
messages: List[ChatSession] = list(
    map_ai_messages(merged_messages, sender="Jiminy Cricket")
)

### 下一步

然后，你可以根据自己的需要使用这些消息，例如微调模型、进行少样本示例选择或直接预测下一条消息。

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

for chunk in llm.stream(messages[0]["messages"]):
    print(chunk.content, end="", flush=True)

I said, "You better trust your conscience."